In [53]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

In [57]:
df=pd.read_csv('train.csv')

In [58]:
df

,Tweets,label
0,Asked #ChatGPT about what it thinks are the pr...,Positive
1,#ChatGPT tornado has already traveled around t...,Neutral
2,This is a great explanation of why #EVs are mo...,Positive
3,‘if you need to write a box-ticking social med...,Positive
4,Just saw an AI tool making my coffee for me. \...,Positive
...,...,...
765,NaN,NaN
766,NaN,NaN
767,NaN,NaN
768,NaN,NaN


In [59]:
df.isnull().sum()

Tweets    171
label     171
dtype: int64

In [60]:
df.head(600)

,Tweets,label
0,Asked #ChatGPT about what it thinks are the pr...,Positive
1,#ChatGPT tornado has already traveled around t...,Neutral
2,This is a great explanation of why #EVs are mo...,Positive
3,‘if you need to write a box-ticking social med...,Positive
4,Just saw an AI tool making my coffee for me. \...,Positive
...,...,...
595,3. KeyboardX also provides a more secure envir...,Positive
596,2. KeyboardX allows users to quickly and easil...,Positive
597,"I asked ChatGPT why the app I’m working on, Ke...",Positive
598,There's been a lot of talk about AI engines an...,Positive


In [61]:
df=df.dropna()

In [62]:
df.isnull().sum()

Tweets    0
label     0
dtype: int64

In [63]:
df['Tweets'][15]

'Confidence can even make the truth seem false XD\n#ChatGPT https://t.co/VQ7lf5bhhI'

In [64]:
df['Tweets'][15]

'Confidence can even make the truth seem false XD\n#ChatGPT https://t.co/VQ7lf5bhhI'

In [65]:
df['Tweets'][15]

'Confidence can even make the truth seem false XD\n#ChatGPT https://t.co/VQ7lf5bhhI'

In [66]:
for i in range(599):
    df['Tweets'][i] = re.sub(r"https:\S+|www\S+|https\S+", '', df['Tweets'][i])
    df['Tweets'][i] = re.sub(r'@[\w]*', '', df['Tweets'][i])
    df['Tweets'][i] = re.sub(r'\W', ' ', df['Tweets'][i])
    df['Tweets'][i] = re.sub(r'\d', ' ', df['Tweets'][i])

C:\Users\hp\AppData\Local\Temp\ipykernel_6904\1336326747.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tweets'][i] = re.sub(r"https:\S+|www\S+|https\S+", '', df['Tweets'][i])
C:\Users\hp\AppData\Local\Temp\ipykernel_6904\1336326747.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tweets'][i] = re.sub(r'@[\w]*', '', df['Tweets'][i])
C:\Users\hp\AppData\Local\Temp\ipykernel_6904\1336326747.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [67]:
df['Tweets'][15]

'Confidence can even make the truth seem false XD  ChatGPT '

In [68]:
for i in range(599):
    df['Tweets'][i]=(df['Tweets'][i]).lower()

C:\Users\hp\AppData\Local\Temp\ipykernel_6904\2297265130.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tweets'][i]=(df['Tweets'][i]).lower()


In [69]:
for i in range(599):
    df['Tweets'][i] = word_tokenize(df['Tweets'][i])

C:\Users\hp\AppData\Local\Temp\ipykernel_6904\3433724338.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tweets'][i] = word_tokenize(df['Tweets'][i])


In [70]:
df['tweet_tokens']=None
df

C:\Users\hp\AppData\Local\Temp\ipykernel_6904\3190894782.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_tokens']=None


,Tweets,label,tweet_tokens
0,"[asked, chatgpt, about, what, it, thinks, are,...",Positive,None
1,"[chatgpt, tornado, has, already, traveled, aro...",Neutral,None
2,"[this, is, a, great, explanation, of, why, evs...",Positive,None
3,"[if, you, need, to, write, a, box, ticking, so...",Positive,None
4,"[just, saw, an, ai, tool, making, my, coffee, ...",Positive,None
...,...,...,...
594,"[gpt, s, ai, capabilities, are, also, more, ac...",Positive,None
595,"[keyboardx, also, provides, a, more, secure, e...",Positive,None
596,"[keyboardx, allows, users, to, quickly, and, e...",Positive,None
597,"[i, asked, chatgpt, why, the, app, i, m, worki...",Positive,None


In [71]:
stop_words = set(stopwords.words('english'))

In [72]:
lemmatizer = WordNetLemmatizer()

In [73]:
tokens=[]
for i in range(599):
    for token in df['Tweets'][i]:
        if token not in stop_words:
            tokens.append(token)
    df['tweet_tokens'][i]=tokens
    tokens=[]

C:\Users\hp\AppData\Local\Temp\ipykernel_6904\2618934204.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_tokens'][i]=tokens


In [74]:
df

,Tweets,label,tweet_tokens
0,"[asked, chatgpt, about, what, it, thinks, are,...",Positive,"[asked, chatgpt, thinks, problems, associated,..."
1,"[chatgpt, tornado, has, already, traveled, aro...",Neutral,"[chatgpt, tornado, already, traveled, around, ..."
2,"[this, is, a, great, explanation, of, why, evs...",Positive,"[great, explanation, evs, efficient, energy, s..."
3,"[if, you, need, to, write, a, box, ticking, so...",Positive,"[need, write, box, ticking, social, media, pos..."
4,"[just, saw, an, ai, tool, making, my, coffee, ...",Positive,"[saw, ai, tool, making, coffee, chatgpt, ai, a..."
...,...,...,...
594,"[gpt, s, ai, capabilities, are, also, more, ac...",Positive,"[gpt, ai, capabilities, also, accessible, keyb..."
595,"[keyboardx, also, provides, a, more, secure, e...",Positive,"[keyboardx, also, provides, secure, environmen..."
596,"[keyboardx, allows, users, to, quickly, and, e...",Positive,"[keyboardx, allows, users, quickly, easily, ac..."
597,"[i, asked, chatgpt, why, the, app, i, m, worki...",Positive,"[asked, chatgpt, app, working, keyboardx, bett..."


In [75]:
for i in range(599):
    tokens=[]
    for token in df['Tweets'][i]:
        token = lemmatizer.lemmatize(token,'v')
        tokens.append(token)
    df['tweet_tokens'][i]=tokens

C:\Users\hp\AppData\Local\Temp\ipykernel_6904\1040707579.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_tokens'][i]=tokens


In [76]:
df

,Tweets,label,tweet_tokens
0,"[asked, chatgpt, about, what, it, thinks, are,...",Positive,"[ask, chatgpt, about, what, it, think, be, the..."
1,"[chatgpt, tornado, has, already, traveled, aro...",Neutral,"[chatgpt, tornado, have, already, travel, arou..."
2,"[this, is, a, great, explanation, of, why, evs...",Positive,"[this, be, a, great, explanation, of, why, evs..."
3,"[if, you, need, to, write, a, box, ticking, so...",Positive,"[if, you, need, to, write, a, box, tick, socia..."
4,"[just, saw, an, ai, tool, making, my, coffee, ...",Positive,"[just, saw, an, ai, tool, make, my, coffee, fo..."
...,...,...,...
594,"[gpt, s, ai, capabilities, are, also, more, ac...",Positive,"[gpt, s, ai, capabilities, be, also, more, acc..."
595,"[keyboardx, also, provides, a, more, secure, e...",Positive,"[keyboardx, also, provide, a, more, secure, en..."
596,"[keyboardx, allows, users, to, quickly, and, e...",Positive,"[keyboardx, allow, users, to, quickly, and, ea..."
597,"[i, asked, chatgpt, why, the, app, i, m, worki...",Positive,"[i, ask, chatgpt, why, the, app, i, m, work, o..."


In [81]:
vectorizer = CountVectorizer()
for i in range(599):
    df['tweet_tokens'][i]=vectorizer.fit_transform(df['tweet_tokens'][i])

C:\Users\hp\AppData\Local\Temp\ipykernel_6904\3570213057.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_tokens'][i]=vectorizer.fit_transform(df['tweet_tokens'][i])


In [82]:
df

,Tweets,label,tweet_tokens
0,"[asked, chatgpt, about, what, it, thinks, are,...",Positive,"(0, 3)\t1\n (1, 7)\t1\n (2, 0)\t1\n (3, 2..."
1,"[chatgpt, tornado, has, already, traveled, aro...",Neutral,"(0, 6)\t1\n (1, 32)\t1\n (2, 13)\t1\n (3,..."
2,"[this, is, a, great, explanation, of, why, evs...",Positive,"(0, 19)\t1\n (1, 1)\t1\n (3, 11)\t1\n (4,..."
3,"[if, you, need, to, write, a, box, ticking, so...",Positive,"(0, 11)\t1\n (1, 29)\t1\n (2, 15)\t1\n (3..."
4,"[just, saw, an, ai, tool, making, my, coffee, ...",Positive,"(0, 6)\t1\n (1, 10)\t1\n (2, 2)\t1\n (3, ..."
...,...,...,...
594,"[gpt, s, ai, capabilities, are, also, more, ac...",Positive,"(0, 11)\t1\n (2, 2)\t1\n (3, 8)\t1\n (4, ..."
595,"[keyboardx, also, provides, a, more, secure, e...",Positive,"(0, 11)\t1\n (1, 1)\t1\n (2, 15)\t1\n (4,..."
596,"[keyboardx, allows, users, to, quickly, and, e...",Positive,"(0, 10)\t1\n (1, 2)\t1\n (2, 16)\t1\n (3,..."
597,"[i, asked, chatgpt, why, the, app, i, m, worki...",Positive,"(1, 3)\t1\n (2, 11)\t1\n (3, 31)\t1\n (4,..."


In [83]:
lb=LabelEncoder()

In [84]:
df['label']=lb.fit_transform(df['label'])

C:\Users\hp\AppData\Local\Temp\ipykernel_6904\4275040492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label']=lb.fit_transform(df['label'])


In [85]:
df

,Tweets,label,tweet_tokens
0,"[asked, chatgpt, about, what, it, thinks, are,...",2,"(0, 3)\t1\n (1, 7)\t1\n (2, 0)\t1\n (3, 2..."
1,"[chatgpt, tornado, has, already, traveled, aro...",1,"(0, 6)\t1\n (1, 32)\t1\n (2, 13)\t1\n (3,..."
2,"[this, is, a, great, explanation, of, why, evs...",2,"(0, 19)\t1\n (1, 1)\t1\n (3, 11)\t1\n (4,..."
3,"[if, you, need, to, write, a, box, ticking, so...",2,"(0, 11)\t1\n (1, 29)\t1\n (2, 15)\t1\n (3..."
4,"[just, saw, an, ai, tool, making, my, coffee, ...",2,"(0, 6)\t1\n (1, 10)\t1\n (2, 2)\t1\n (3, ..."
...,...,...,...
594,"[gpt, s, ai, capabilities, are, also, more, ac...",2,"(0, 11)\t1\n (2, 2)\t1\n (3, 8)\t1\n (4, ..."
595,"[keyboardx, also, provides, a, more, secure, e...",2,"(0, 11)\t1\n (1, 1)\t1\n (2, 15)\t1\n (4,..."
596,"[keyboardx, allows, users, to, quickly, and, e...",2,"(0, 10)\t1\n (1, 2)\t1\n (2, 16)\t1\n (3,..."
597,"[i, asked, chatgpt, why, the, app, i, m, worki...",2,"(1, 3)\t1\n (2, 11)\t1\n (3, 31)\t1\n (4,..."


In [106]:
x=df['tweet_tokens'].values

In [87]:
y=df['label'].values

In [88]:
X_train_full,X_test,y_train_full,y_test=train_test_split(x,y,random_state=42)

In [89]:
X_train,X_valid,y_train,y_valid=train_test_split(X_train_full,y_train_full,random_state=42)

In [90]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=32, input_length=20),
    LSTM(units=100),
    Dense(3, activation='softmax')  
])

In [94]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 32)            160000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               53200     
                                                                 
 dense_1 (Dense)             (None, 3)                 303       
                                                                 
Total params: 213503 (834.00 KB)
Trainable params: 213503 (834.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [100]:
from keras.utils import to_categorical


X_train_dense = np.array(X_train)
y_train_one_hot = to_categorical(y_train)



In [101]:
model.fit(X_train,y_train_one_hot, epochs=20, batch_size=64)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type csr_matrix).